# 16IT146 &emsp; &emsp;  &emsp;  &emsp;  &emsp;  &emsp;    IT441: Data Science

# WEB SCRAPING : moneycontrol

In [1]:
import numpy as np # linear algebra
import pandas as pd # pandas for dataframe based data processing and CSV file I/O
import requests # for http requests
from bs4 import BeautifulSoup # for html parsing and scraping
import bs4
from fastnumbers import isfloat 
from fastnumbers import fast_float
from multiprocessing.dummy import Pool as ThreadPool 

import matplotlib.pyplot as plt
import seaborn as sns
import json
from tidylib import tidy_document # for tidying incorrect html

sns.set_style('whitegrid')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def ffloat(string):
    if string is None:
        return np.nan
    if type(string)==float or type(string)==np.float64:
        return string
    if type(string)==int or type(string)==np.int64:
        return string
    return fast_float(string.split(" ")[0].replace(',','').replace('%',''),
                      default=np.nan)

In [3]:
def ffloat_list(string_list):
    return list(map(ffloat,string_list))

In [4]:
def remove_multiple_spaces(string):
    if type(string)==str:
        return ' '.join(string.split())
    return string

In [5]:
url = "https://www.moneycontrol.com/commodity/crudeoil-price.html#19sep2019"
from IPython.core.display import HTML
HTML("<b>Rendered HTML</b>")

In [6]:
page_response = requests.get("https://www.moneycontrol.com/india/stockpricequote/auto-cars-jeeps/marutisuzukiindia/MS24", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")
HTML(str(page_content.find("h1")))

HTML(str(page_content.find("div",attrs={'id':"div_bse_nse_livebox_wrap"})))

In [7]:
page_response = requests.get("https://www.moneycontrol.com/mutual-funds/performance-tracker/returns/large-cap-fund.html", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")

HTML(str(page_content.find("table",attrs={'id':"dataTableId"})))
get_table_simple(page_content)

Scheme Name,Plan,Category Name,Crisil Rank,AuM (Cr),1W,1M,3M,6M,YTD,1Y,2Y,3Y,5Y,10Y
Axis Bluechip Fund - Direct Plan - GrowthLarge Cap Fund,Direct Plan,Large Cap Fund,5,"7,005.10",-0.44%,7.06%,4.11%,10.68%,10.68%,18.61%,16.05%,16.73%,13.15%,-
Axis Bluechip Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,5,"7,005.10",-0.48%,6.94%,3.80%,9.97%,9.97%,17.12%,14.50%,15.24%,11.81%,-
BNP Paribas Large Cap Fund - Direct Plan - GrowthLarge Cap Fund,Direct Plan,Large Cap Fund,3,747.88,0.09%,5.72%,2.99%,8.15%,8.15%,16.68%,9.78%,11.66%,10.85%,-
BNP Paribas Large Cap Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,3,747.88,0.06%,5.61%,2.67%,7.53%,7.53%,15.29%,8.33%,10.22%,9.53%,11.83%
LIC MF Large Cap Fund - Direct Plan - GrowthLarge Cap Fund,Direct Plan,Large Cap Fund,3,279.55,-0.93%,6.18%,1.66%,6.53%,6.53%,14.14%,8.58%,9.88%,9.47%,-
LIC MF Large Cap Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,3,279.55,-0.95%,6.10%,1.40%,5.97%,5.97%,12.95%,7.36%,8.66%,8.35%,8.85%
IDBI India Top 100 Equity Fund - Direct Plan - GrowthLarge Cap Fund,Direct Plan,Large Cap Fund,1,351.16,-0.56%,5.81%,2.15%,4.32%,4.32%,12.79%,4.60%,7.71%,9.17%,-
IDBI India Top 100 Equity Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,1,351.16,-0.60%,5.69%,1.81%,3.56%,3.56%,11.09%,2.82%,5.91%,7.81%,-
Canara Robeco Bluechip Equity Fund - Direct Plan - GrowthLarge Cap Fund,Direct Plan,Large Cap Fund,5,220.60,-0.58%,5.86%,0.59%,4.00%,4.00%,10.91%,10.51%,11.91%,10.87%,-
DSP Top 100 Equity Fund - Direct Plan - GrowthLarge Cap Fund,Direct Plan,Large Cap Fund,2,"2,316.70",-1.24%,8.40%,-0.32%,2.71%,2.71%,10.59%,5.39%,7.54%,8.34%,-


NameError: name 'get_table_simple' is not defined

In [8]:
from urllib.request import urlopen
htmlx = urlopen("https://www.moneycontrol.com/mutual-funds/performance-tracker/returns/large-cap-fund.html")
soup = BeautifulSoup(htmlx, 'lxml')
rows = soup.find_all('tr')
print(rows[:10])

[<tr><td><link href="http://www.moneycontrol.com/rss/latestnews.xml" rel="alternate" title="MoneyControl.com News" type="application/rss+xml"/></td></tr>, <tr>
<th>Scheme Name
                                            <div class="filter_arw"><span class="filarw_up"></span> <span class="filarw_dwn"></span></div>
</th>
<th style="display: none;">Plan
                                            <div class="filter_arw"><span class="filarw_up"></span> <span class="filarw_dwn"></span></div>
</th>
<th style="display: none;">Category Name
                                            <div class="filter_arw"><span class="filarw_up"></span> <span class="filarw_dwn"></span></div>
</th>
<th class="crisil_col">Crisil Rank
                                            <div class="filter_arw"><span class="filarw_up"></span> <span class="filarw_dwn"></span></div>
</th>
<th>AuM (Cr)
                                            <div class="filter_arw"><span class="filarw_up"></span> <span class="filarw_dwn

In [9]:
for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)

[<td><div><span class="SpAd">Sponsored Adv</span><a class="startsip" href="http://pubads.g.doubleclick.net/gampad/clk?id=4720299968&amp;iu=/1039154/Moneycontrol/MC_Click_Tracker" target="_blank" title="Start SIP">Start SIP</a></div><a class="robo_medium" href="http://pubads.g.doubleclick.net/gampad/clk?id=4720299968&amp;iu=/1039154/Moneycontrol/MC_Click_Tracker" target="_blank">ICICI Prudential Bluechip Fund - Direct Plan - Growth<span class="sn">Large Cap Fund</span></a></td>, <td style="display: none;">Direct Plan</td>, <td style="display: none;">Direct Plan</td>, <td class="crisil_col" data-order="3"><span class="rank_block">3</span></td>, <td>21,672.64</td>, <td class="tbl_redtxt">-0.60%</td>, <td class="tbl_greentxt">4.88%</td>, <td class="tbl_redtxt">-1.90%</td>, <td class="tbl_greentxt">0.58%</td>, <td class="tbl_greentxt">0.58%</td>, <td class="tbl_greentxt">5.20%</td>, <td class="tbl_greentxt">7.18%</td>, <td class="tbl_greentxt">10.88%</td>, <td class="tbl_greentxt">10.50%</t

bs4.element.ResultSet

In [10]:
str_cells = str(row_td)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
print(cleantext)

[Sponsored AdvStart SIPICICI Prudential Bluechip Fund - Direct Plan - GrowthLarge Cap Fund, Direct Plan, Direct Plan, 3, 21,672.64, -0.60%, 4.88%, -1.90%, 0.58%, 0.58%, 5.20%, 7.18%, 10.88%, 10.50%, -]


In [11]:
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

[Sponsored AdvStart SIPICICI Prudential Bluechip Fund - Direct Plan - GrowthLarge Cap Fund, Direct Plan, Direct Plan, 3, 21,672.64, -0.60%, 4.88%, -1.90%, 0.58%, 0.58%, 5.20%, 7.18%, 10.88%, 10.50%, -]


str

In [12]:
df = pd.DataFrame(list_rows)
df.head(10)

,0
0,[]
1,[]
2,[Axis Bluechip Fund - Direct Plan - GrowthLarg...
3,"[Axis Bluechip Fund - GrowthLarge Cap Fund, Re..."
4,[BNP Paribas Large Cap Fund - Direct Plan - Gr...
5,[BNP Paribas Large Cap Fund - GrowthLarge Cap ...
6,[LIC MF Large Cap Fund - Direct Plan - GrowthL...
7,"[LIC MF Large Cap Fund - GrowthLarge Cap Fund,..."
8,[IDBI India Top 100 Equity Fund - Direct Plan ...
9,[IDBI India Top 100 Equity Fund - GrowthLarge ...


In [13]:
df1 = df[0].str.split(',', expand=True)
df1.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,[Axis Bluechip Fund - Direct Plan - GrowthLarg...,Direct Plan,Large Cap Fund,5,7,005.10,-0.44%,7.06%,4.11%,10.68%,10.68%,18.61%,16.05%,16.73%,13.15%,-]
3,[Axis Bluechip Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,5,7,005.10,-0.48%,6.94%,3.80%,9.97%,9.97%,17.12%,14.50%,15.24%,11.81%,-]
4,[BNP Paribas Large Cap Fund - Direct Plan - Gr...,Direct Plan,Large Cap Fund,3,747.88,0.09%,5.72%,2.99%,8.15%,8.15%,16.68%,9.78%,11.66%,10.85%,-],None
5,[BNP Paribas Large Cap Fund - GrowthLarge Cap ...,Regular,Large Cap Fund,3,747.88,0.06%,5.61%,2.67%,7.53%,7.53%,15.29%,8.33%,10.22%,9.53%,11.83%],None
6,[LIC MF Large Cap Fund - Direct Plan - GrowthL...,Direct Plan,Large Cap Fund,3,279.55,-0.93%,6.18%,1.66%,6.53%,6.53%,14.14%,8.58%,9.88%,9.47%,-],None
7,[LIC MF Large Cap Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,3,279.55,-0.95%,6.10%,1.40%,5.97%,5.97%,12.95%,7.36%,8.66%,8.35%,8.85%],None
8,[IDBI India Top 100 Equity Fund - Direct Plan ...,Direct Plan,Large Cap Fund,1,351.16,-0.56%,5.81%,2.15%,4.32%,4.32%,12.79%,4.60%,7.71%,9.17%,-],None
9,[IDBI India Top 100 Equity Fund - GrowthLarge ...,Regular,Large Cap Fund,1,351.16,-0.60%,5.69%,1.81%,3.56%,3.56%,11.09%,2.82%,5.91%,7.81%,-],None


In [14]:
df1[0] = df1[0].str.strip('[')
df1.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Axis Bluechip Fund - Direct Plan - GrowthLarge...,Direct Plan,Large Cap Fund,5,7,005.10,-0.44%,7.06%,4.11%,10.68%,10.68%,18.61%,16.05%,16.73%,13.15%,-]
3,Axis Bluechip Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,5,7,005.10,-0.48%,6.94%,3.80%,9.97%,9.97%,17.12%,14.50%,15.24%,11.81%,-]
4,BNP Paribas Large Cap Fund - Direct Plan - Gro...,Direct Plan,Large Cap Fund,3,747.88,0.09%,5.72%,2.99%,8.15%,8.15%,16.68%,9.78%,11.66%,10.85%,-],None
5,BNP Paribas Large Cap Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,3,747.88,0.06%,5.61%,2.67%,7.53%,7.53%,15.29%,8.33%,10.22%,9.53%,11.83%],None
6,LIC MF Large Cap Fund - Direct Plan - GrowthLa...,Direct Plan,Large Cap Fund,3,279.55,-0.93%,6.18%,1.66%,6.53%,6.53%,14.14%,8.58%,9.88%,9.47%,-],None
7,LIC MF Large Cap Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,3,279.55,-0.95%,6.10%,1.40%,5.97%,5.97%,12.95%,7.36%,8.66%,8.35%,8.85%],None
8,IDBI India Top 100 Equity Fund - Direct Plan -...,Direct Plan,Large Cap Fund,1,351.16,-0.56%,5.81%,2.15%,4.32%,4.32%,12.79%,4.60%,7.71%,9.17%,-],None
9,IDBI India Top 100 Equity Fund - GrowthLarge C...,Regular,Large Cap Fund,1,351.16,-0.60%,5.69%,1.81%,3.56%,3.56%,11.09%,2.82%,5.91%,7.81%,-],None


In [15]:
col_labels = soup.find_all('th')

In [16]:
all_header = []
col_str = str(col_labels)
cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
all_header.append(cleantext2)
length = len(all_header)
for i in range(length): 
    all_header[i]=all_header[i].replace('\n', '')
print(all_header)

['[Scheme Name                                             , Plan                                             , Category Name                                             , Crisil Rank                                             , AuM (Cr)                                             , 1W                                             , 1M                                             , 3M                                             , 6M                                             , YTD                                             , 1Y                                             , 2Y                                             , 3Y                                             , 5Y                                             , 10Y                                             ]']


In [17]:
df2 = pd.DataFrame(all_header)
df2.head()

,0
0,[Scheme Name ...


In [18]:
df3 = df2[0].str.split(',', expand=True)
df3.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,[Scheme Name ...,Plan ...,Category Name ...,Crisil Rank ...,AuM (Cr) ...,1W,1M,3M,6M,YTD,1Y,2Y,3Y,5Y,10Y ...


In [19]:
frames = [df3, df1]

df4 = pd.concat(frames)
df4.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,[Scheme Name ...,Plan ...,Category Name ...,Crisil Rank ...,AuM (Cr) ...,1W,1M,3M,6M,YTD,1Y,2Y,3Y,5Y,10Y ...,NaN
0,],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Axis Bluechip Fund - Direct Plan - GrowthLarge...,Direct Plan,Large Cap Fund,5,7,005.10,-0.44%,7.06%,4.11%,10.68%,10.68%,18.61%,16.05%,16.73%,13.15%,-]
3,Axis Bluechip Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,5,7,005.10,-0.48%,6.94%,3.80%,9.97%,9.97%,17.12%,14.50%,15.24%,11.81%,-]
4,BNP Paribas Large Cap Fund - Direct Plan - Gro...,Direct Plan,Large Cap Fund,3,747.88,0.09%,5.72%,2.99%,8.15%,8.15%,16.68%,9.78%,11.66%,10.85%,-],None
5,BNP Paribas Large Cap Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,3,747.88,0.06%,5.61%,2.67%,7.53%,7.53%,15.29%,8.33%,10.22%,9.53%,11.83%],None
6,LIC MF Large Cap Fund - Direct Plan - GrowthLa...,Direct Plan,Large Cap Fund,3,279.55,-0.93%,6.18%,1.66%,6.53%,6.53%,14.14%,8.58%,9.88%,9.47%,-],None
7,LIC MF Large Cap Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,3,279.55,-0.95%,6.10%,1.40%,5.97%,5.97%,12.95%,7.36%,8.66%,8.35%,8.85%],None
8,IDBI India Top 100 Equity Fund - Direct Plan -...,Direct Plan,Large Cap Fund,1,351.16,-0.56%,5.81%,2.15%,4.32%,4.32%,12.79%,4.60%,7.71%,9.17%,-],None


In [20]:
df5 = df4.rename(columns=df4.iloc[0])
df5.head()

,[Scheme Name,Plan,Category Name,Crisil Rank,AuM (Cr),1W,1M,3M,6M,YTD,1Y,2Y,3Y,5Y,10Y ],nan
0,[Scheme Name ...,Plan ...,Category Name ...,Crisil Rank ...,AuM (Cr) ...,1W,1M,3M,6M,YTD,1Y,2Y,3Y,5Y,10Y ...,NaN
0,],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Axis Bluechip Fund - Direct Plan - GrowthLarge...,Direct Plan,Large Cap Fund,5,7,005.10,-0.44%,7.06%,4.11%,10.68%,10.68%,18.61%,16.05%,16.73%,13.15%,-]
3,Axis Bluechip Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,5,7,005.10,-0.48%,6.94%,3.80%,9.97%,9.97%,17.12%,14.50%,15.24%,11.81%,-]


In [21]:
df6 = df5.dropna(axis=0, how='any')

In [22]:
df7 = df6.drop(df6.index[0])
df7.head()

,[Scheme Name,Plan,Category Name,Crisil Rank,AuM (Cr),1W,1M,3M,6M,YTD,1Y,2Y,3Y,5Y,10Y ],nan
3,Axis Bluechip Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,5,7,005.10,-0.48%,6.94%,3.80%,9.97%,9.97%,17.12%,14.50%,15.24%,11.81%,-]
11,DSP Top 100 Equity Fund - Direct Plan - Growth...,Direct Plan,Large Cap Fund,2,2,316.70,-1.24%,8.40%,-0.32%,2.71%,2.71%,10.59%,5.39%,7.54%,8.34%,-]
12,DSP Top 100 Equity Fund - Regular Plan - Growt...,Regular,Large Cap Fund,3,2,316.70,-1.25%,8.34%,-0.49%,2.36%,2.36%,9.84%,4.67%,6.80%,7.62%,9.31%]
14,SBI Blue Chip Fund - Direct Plan - GrowthLarge...,Direct Plan,Large Cap Fund,3,21,483.48,-0.77%,5.63%,-1.20%,2.25%,2.25%,9.75%,6.54%,8.60%,11.22%,-]
17,SBI Blue Chip Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,3,21,483.48,-0.79%,5.56%,-1.40%,1.88%,1.88%,8.92%,5.55%,7.53%,10.11%,11.30%]


In [23]:
df7.rename(columns={'[Scheme Name ': 'Scheme Name'},inplace=True)
df7.rename(columns={'10Y ]': '10Y'},inplace=True)
df7.head()

,[Scheme Name,Plan,Category Name,Crisil Rank,AuM (Cr),1W,1M,3M,6M,YTD,1Y,2Y,3Y,5Y,10Y ],nan
3,Axis Bluechip Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,5,7,005.10,-0.48%,6.94%,3.80%,9.97%,9.97%,17.12%,14.50%,15.24%,11.81%,-]
11,DSP Top 100 Equity Fund - Direct Plan - Growth...,Direct Plan,Large Cap Fund,2,2,316.70,-1.24%,8.40%,-0.32%,2.71%,2.71%,10.59%,5.39%,7.54%,8.34%,-]
12,DSP Top 100 Equity Fund - Regular Plan - Growt...,Regular,Large Cap Fund,3,2,316.70,-1.25%,8.34%,-0.49%,2.36%,2.36%,9.84%,4.67%,6.80%,7.62%,9.31%]
14,SBI Blue Chip Fund - Direct Plan - GrowthLarge...,Direct Plan,Large Cap Fund,3,21,483.48,-0.77%,5.63%,-1.20%,2.25%,2.25%,9.75%,6.54%,8.60%,11.22%,-]
17,SBI Blue Chip Fund - GrowthLarge Cap Fund,Regular,Large Cap Fund,3,21,483.48,-0.79%,5.56%,-1.40%,1.88%,1.88%,8.92%,5.55%,7.53%,10.11%,11.30%]


In [25]:
page_response = requests.get("https://www.moneycontrol.com/india/stockpricequote/transportlogistics/containercorporationindia/CCI", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")
HTML(str(page_content.find("h1")))

HTML(str(page_content.find("div",attrs={'id':"div_bse_nse_livebox_wrap"})))

In [26]:
page_response = requests.get("https://www.moneycontrol.com/india/stockpricequote/infrastructure-general/siemens/S", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")
HTML(str(page_content.find("h1")))

HTML(str(page_content.find("div",attrs={'id':"div_bse_nse_livebox_wrap"})))

In [27]:
page_response = requests.get("https://www.moneycontrol.com/india/stockpricequote/constructioncontracting-civil/patelengineeringcompany/PEC", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")
HTML(str(page_content.find("h1")))

HTML(str(page_content.find("div",attrs={'id':"div_bse_nse_livebox_wrap"})))

In [38]:
page_response = requests.get("https://www.moneycontrol.com/india/stockpricequote/electric-equipment/denoraindia/DNI", timeout=240)
page_content = BeautifulSoup(page_response.content, "html.parser")
HTML(str(page_content.find("h1")))
HTML(str(page_content.find("div",attrs={'id':"standalone_valuation"})))

<b>Conclusion:</b> In this practical I have performed web scraping of 5 different stock data and provided some data for data pre-processing.